# Prototype hybrid embedding : data-parallel frequent categories and model- parallel infrequent categories

In [ ]:
import numpy as np
from copy import deepcopy

np.random.seed(42) # guarantees the same data each execution

In [ ]:
batch_size = 64
num_slots = 10
num_nodes = 2
num_networks_per_node = 4
embedding_vec_size = 128
num_networks = num_networks_per_node*num_nodes
local_batch_size = int((batch_size + num_networks - 1)/(num_networks))

In [ ]:
def flatten_data(data):
    # concatenate all iterations
    samples_data = np.concatenate([deepcopy(data[i][1])
                                   for i in range(len(data))], axis=1)

    # data dimensions
    embedding_sizes = data[0][0]
    num_tables = samples_data.shape[0]
    num_samples = samples_data.shape[1]

    # flatten and make categories unique
    samples = np.zeros(num_tables * num_samples, dtype=np.int32)
    category_index_offset = 0
    for j in range(num_tables):
        for i in range(num_samples):
            samples[i * num_tables + j] = (category_index_offset
                                          + samples_data[j, i])
        category_index_offset += embedding_sizes[j]

    return samples

In [ ]:
# Generate synthetic data
embed_sizes = np.random.randint(1, 2*batch_size, num_slots);
num_categories = sum(embed_sizes)
print("num_categories:", num_categories)

In [ ]:
print(",".join(map(str, embed_sizes)))

In [ ]:
data = []
num_batches = 15
for i in range(num_batches):
    batch = np.zeros((num_slots, batch_size))
    for j in range(num_slots):
        batch[j, :] = np.random.randint(0, embed_sizes[j], batch_size)
    data.append((embed_sizes, batch))

In [ ]:
samples = flatten_data(data)

In [ ]:
class Gpu:

    def __init__(self):
        self.frequent_categories = None
        self.category_frequent_index = None
        self.frequent_sample_indices = None
        self.frequent_embedding_vectors = None
        self.frequent_partial_gradients = None
        
        self.num_infrequent = None
        self.infrequent_embedding_vectors = None
        self.category_location = None
        self.model_indices = None
        self.model_indices_offsets = None
        self.network_indices = None
        self.network_indices_offsets = None
        self.ggpu_id = None
        self.gpu_id = None
        self.node = None

        self.a2a_fwd_message_buf = []

    def init_frequent_embedding_cache(self, num_frequent, embedding_vec_size):
        self.num_frequent = num_frequent
        #self.frequent_embedding_vectors = np.random.random((num_frequent,embedding_vec_size)) + 2
        #self.frequent_partial_gradients = np.random.random((num_frequent,embedding_vec_size))
        self.frequent_embedding_vectors = np.zeros((num_frequent,embedding_vec_size), dtype = np.float32)
        self.frequent_partial_gradients = np.zeros((num_frequent,embedding_vec_size), dtype = np.float32)
        for i in range(num_frequent):
            for j in range(embedding_vec_size):
                self.frequent_embedding_vectors[i, j] = float(i*embedding_vec_size + j + 1);
    def init_infrequent_embedding_cache(self, embedding_vec_size):
        self.num_infrequent = np.sum(self.category_location[:,0] == self.ggpu_id)
        #self.infrequent_embedding_vectors = np.random.random((self.num_infrequent,embedding_vec_size))
        self.infrequent_embedding_vectors = np.zeros((self.num_infrequent,embedding_vec_size), dtype = np.float32)
        for i in range(self.num_infrequent):
            for j in range(embedding_vec_size):
                self.infrequent_embedding_vectors[i, j] = -1.0*float(i*embedding_vec_size + j + 1)

    def init_embedding_output(self, embedding_vec_size):
        self.embedding_output = np.zeros((int(local_batch_size*num_slots), embedding_vec_size), dtype = np.float32)
        
class Node:

    def __init__(self, num_gpus):
        self.gpus = [Gpu() for i in range(num_gpus)]
        for i in range(num_gpus):
            self.gpus[i].gpu_id = i
            self.gpus[i].node = self # reference to this node

class Network:

    def __init__(self, nodes):
        self.nodes = nodes

    def all_reduce(self):
        pass

    def all_to_all(self, samples):
        gpus = [gpu for node in self.nodes for gpu in node.gpus]
        for srcidx,srcgpu in enumerate(gpus):
            for dstidx, dstgpu in enumerate(gpus):
                model_indices_to_dst = srcgpu.model_indices[srcgpu.model_indices_offsets[dstidx]:srcgpu.model_indices_offsets[dstidx+1]]
                samples_to_dst = samples[model_indices_to_dst]
                embedding_vec_incides = srcgpu.category_location[samples_to_dst, 1]
                data_send = srcgpu.infrequent_embedding_vectors[embedding_vec_incides, :]
                dstgpu.a2a_fwd_message_buf.append(data_send)
        for gpu in gpus:
            gpu.a2a_fwd_message_buf = np.concatenate(gpu.a2a_fwd_message_buf)

    def embedding_network_forward(self, samples):
        gpus = [gpu for node in self.nodes for gpu in node.gpus]
        for gpuid, gpu in enumerate(gpus):
            for i, idx in enumerate(gpu.network_indices):
                gpu.embedding_output[idx] = gpu.a2a_fwd_message_buf[i]
            index_base = int(local_batch_size*num_slots*gpu.ggpu_id)
            for i, idx in enumerate(gpu.frequent_sample_indices):
                category = samples[idx + index_base]
                embedding_vec_index = gpu_.category_frequent_index[category]
                gpu.embedding_output[idx] = gpu_.frequent_embedding_vectors[embedding_vec_index]
            num_categories = gpu.category_location.shape[0]
            for i in range(gpu.embedding_output.shape[0]):
                category = samples[i + index_base]
                is_frequent = (gpu.category_location[category][0] == num_categories)
                #print("i = {}, is_frequent = {}, category = {}, location = {}, vec[0] = {}".format(i, is_frequent, category, gpu.category_location[category][0], gpu.embedding_output[i][0]))
                assert is_frequent == (gpu.embedding_output[i][0] > 0), "wrong location of embedding vector"
            #pdb.set_trace()

In [ ]:
nodes = [Node(num_networks_per_node) for i in range(num_nodes)]
gpus = [gpu for node in nodes for gpu in node.gpus]
num_gpus = len(gpus)
for i in range(num_nodes):
    nodes[i].node_id = i
network = Network(nodes)

In [ ]:
num_frequent = 128
uniques, counts = np.unique(samples, return_counts=True)
sorted_uniques = sorted(zip(counts, uniques), key=lambda x: -x[0])
# print(sorted_uniques)
frequent = set([unique_ for _, unique_ in sorted_uniques[:128]])
assert(len(frequent) == num_frequent)
print(num_frequent, "/", num_categories)

In [ ]:
# category_frequent_index
frequent_mask = np.zeros(num_categories, dtype=bool)
for c in frequent:
    frequent_mask[c] = True
category_frequent_index = num_categories * np.ones(num_categories, dtype=np.int32)
category_frequent_index[frequent_mask] = range(num_frequent)

In [ ]:
# category_location
num_infrequent = num_categories - num_frequent
category_location = num_categories * np.ones((num_categories,2), dtype=np.int32)
infrequent_index = np.zeros(num_categories)
infrequent_mask = np.logical_not(frequent_mask)
infrequent_index[infrequent_mask] = range(num_infrequent)
for c in range(num_categories):
    if c not in frequent:
        index = infrequent_index[c]
        category_location[c,:] = [index % num_gpus, index // num_gpus]

In [ ]:
for idx, gpu in enumerate(gpus):
    gpu.category_location = category_location
    gpu.ggpu_id = idx;
    gpu.category_frequent_index = category_frequent_index
    gpu.init_frequent_embedding_cache(num_frequent, embedding_vec_size)
    gpu.init_infrequent_embedding_cache(embedding_vec_size)
    gpu.init_embedding_output(embedding_vec_size)

In [ ]:
n_display = 20
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
print(f'{color.BOLD}{color.RED}category          |-> frequent category cache index{color.END}')
for category in range(n_display):
    frequent_category_cache_index = category_frequent_index[category]
    if frequent_category_cache_index < num_categories:
        print(f'category {color.BOLD}{category:3d}{color.END}      |->  cache index {color.BOLD}{color.BLUE}{frequent_category_cache_index:6,d}{color.END}')
    else:
        print(f'category {color.BOLD}{category:3d}{color.END}      |->  cache index    {color.BOLD}{color.RED}END{color.END}')

In [ ]:
n_display = 20
print(f'{color.BOLD}{color.RED}category          |->  category location {color.END}')
for category in range(n_display):
    location = category_location[category,:]
    if location[0] < num_categories:
        print(f'category {color.BOLD}{category:3d}{color.END}      |->  category location {color.BOLD}{color.GREEN}{location}{color.END}')
    else:
        print(f'category {color.BOLD}{category:3d}{color.END}      |->  category location   {color.BOLD}{color.RED}END{color.END}')

In [ ]:
assert np.sum(category_location[:,0] != num_categories) == num_infrequent
assert np.sum(category_frequent_index != num_categories) == num_frequent

# Index calculations

In [ ]:
from bisect import bisect_left

def get_node_gpu(node_id, gpu_id):
    # not efficient, but that's not the point here! :P
    node = None
    gpu = None
    for node_ in nodes:
        if node_.node_id == node_id:
            node = node_
            break
    for gpu_ in node.gpus:
        if gpu_.gpu_id == gpu_id:
            gpu = gpu_
            break
    return node, gpu

def get_network_id(node_id, gpu_id):
    for i in range(len(gpus)):
        if gpus[i].node.node_id == node_id and gpus[i].gpu_id == gpu_id:
            return i
    raise KeyError(f"Not found: node {node_id}, GPU {gpu_id}")

def cub_DeviceSelect(gpu, samples, network_id):
    location = gpu.category_location[samples,:]
    samples_mask = (location[:,0] == network_id)
    samples_filter = np.r_[:samples.size][samples_mask]
    return samples_filter

# model indices: forward-send, backward-receive
def calculate_model_indices(samples, node_id, gpu_id):
    _, gpu = get_node_gpu(node_id, gpu_id)
    network_id = get_network_id(node_id, gpu_id)
    section_size = samples.size // num_gpus

    sample_model_indices = cub_DeviceSelect(gpu, samples, network_id)
    network_offset_model_indices = np.zeros(num_gpus + 1, dtype=np.int32)
    for i in range(num_gpus):
        network_offset_model_indices[i] = bisect_left(sample_model_indices, i * section_size)
    network_offset_model_indices[-1] = sample_model_indices.size

    return sample_model_indices, network_offset_model_indices

# network indices: forward-receive, backward-send
def calculate_network_indices(samples, node_id, gpu_id):
    _, gpu = get_node_gpu(node_id, gpu_id)

    section_size = samples.size // num_gpus
    network_id = get_network_id(node_id, gpu_id)
    start_idx = network_id * section_size
    end_idx = min((network_id + 1) * section_size, samples.size)
    sub_batch = samples[start_idx:end_idx]

    location = gpu.category_location[sub_batch,:]
    samples_mask = location[:,0] < num_categories
    infrequent_indices = deepcopy(np.r_[:sub_batch.size][samples_mask])
    network_indices = deepcopy(location[:, 0][samples_mask])
    sorted_indices = np.array(sorted(zip(network_indices, infrequent_indices),
                                     key=lambda x: x[0]))

    sample_network_offsets = np.zeros(num_gpus + 1, dtype=np.int32)
    if len(network_indices):
        sample_network_indices = sorted_indices[:,1]
        for i in range(num_gpus):
            sample_network_offsets[i] = bisect_left(sorted_indices[:,0], i)
    else:
        sample_network_indices = np.zeros(0)
    sample_network_offsets[-1] = len(network_indices)
    
    return sample_network_indices, sample_network_offsets

In [ ]:
iteration = 0
batch = flatten_data([data[0]])

In [ ]:
model_indices = {}
model_indices_offsets = {}
for node_ in nodes:
    for gpu_ in node_.gpus:
        node_id = node_.node_id
        gpu_id = gpu_.gpu_id
        idx, off = calculate_model_indices(batch, node_id, gpu_id)
        model_indices[(node_id, gpu_id)] = idx
        model_indices_offsets[(node_id, gpu_id)] = off
        gpu_.model_indices = idx
        gpu_.model_indices_offsets = off

# print(model_indices)
# print(model_indices_offsets)

network_indices = {}
network_indices_offsets = {}
for node_ in nodes:
    for gpu_ in node_.gpus:
        node_id = node_.node_id
        gpu_id = gpu_.gpu_id
        idx, off = calculate_network_indices(batch, node_id, gpu_id)
        network_indices[(node_id, gpu_id)] = idx
        network_indices_offsets[(node_id, gpu_id)] = off
        gpu_.network_indices = idx
        gpu_.network_indices_offsets = off

# print(network_indices)
# print(network_indices_offsets)

In [ ]:
# frequent sample indices
def calculate_frequent_sample_indices(samples, node_id, gpu_id):
    _, gpu = get_node_gpu(node_id, gpu_id)
    
    section_size = samples.size // num_gpus
    network_id = get_network_id(node_id, gpu_id)
    start_idx = network_id * section_size
    end_idx = min((network_id + 1) * section_size, samples.size)
    sub_batch = samples[start_idx:end_idx]

    freq_indices = gpu.category_frequent_index[sub_batch]
    samples_mask = freq_indices < num_categories
    frequent_sample_indices = deepcopy(np.r_[:sub_batch.size][samples_mask])
    
    return frequent_sample_indices

In [ ]:
frequent_sample_indices = {}
for node_ in nodes:
    for gpu_ in node_.gpus:
        node_id = node_.node_id
        gpu_id = gpu_.gpu_id
        frequent_sample_indices[(node_id, gpu_id)] = \
            calculate_frequent_sample_indices(batch, node_id, gpu_id)
        gpu_.frequent_sample_indices = frequent_sample_indices[(node_id, gpu_id)]

# print(frequent_sample_indices)

In [ ]:
network.all_to_all(batch)
network.embedding_network_forward(batch)

In [ ]:
def calculate_model_cache_indices(samples, node_id, gpu_id):
    _, gpu = get_node_gpu(node_id, gpu_id)
    
    section_size = samples.size // num_gpus
    model_id = get_network_id(node_id, gpu_id)
    num_frequent_per_model = num_frequent // num_gpus
    
    # Compute the mask
    network_frequent_mask = np.zeros(num_gpus * num_frequent_per_model, dtype=bool)
    for i in range(num_gpus):
        freq_index = gpu.category_frequent_index[
            samples[i * section_size:(i+1)*section_size]]
        for idx in freq_index:
            if idx < num_frequent and idx // num_frequent_per_model == model_id:
                network_frequent_mask[i * num_frequent_per_model
                                      + idx % num_frequent_per_model] = True
  
    # Select categories according to the mask
    model_cache_indices = np.r_[:num_gpus * num_frequent_per_model][network_frequent_mask]
    
    # Compute offsets
    model_cache_indices_offsets = np.zeros(num_gpus + 1, dtype=np.int32)
    for i in range(num_gpus):
        model_cache_indices_offsets[i] = bisect_left(model_cache_indices, i * num_frequent_per_model)
    model_cache_indices_offsets[num_gpus] = model_cache_indices.size
    
    # Convert to buffer indices
    model_cache_indices = (model_cache_indices % num_frequent_per_model
                           + model_id * num_frequent_per_model)
    
    return model_cache_indices, model_cache_indices_offsets

In [ ]:
model_cache_indices = {}
model_cache_indices_offsets = {}
for node_ in nodes:
    for gpu_ in node_.gpus:
        node_id = node_.node_id
        gpu_id = gpu_.gpu_id
        idx, off = calculate_model_cache_indices(batch, node_id, gpu_id)
        model_cache_indices[(node_id, gpu_id)] = idx
        model_cache_indices_offsets[(node_id, gpu_id)] = off

# print(model_cache_indices)
# print(model_cache_indices_offsets)

In [ ]:
def calculate_network_cache_indices(samples, node_id, gpu_id):
    _, gpu = get_node_gpu(node_id, gpu_id)
    model_id = get_network_id(node_id, gpu_id)
    network_mask = np.zeros(num_frequent, dtype=bool)
    section_size = samples.size // num_gpus
    sample_mlp_batch = samples[model_id * section_size:min(samples.size,(model_id + 1)*section_size)]
    freq_index = gpu.category_frequent_index[sample_mlp_batch]
    for index in freq_index:
        if index < num_frequent:
            network_mask[index] = True
    network_cache_indices = np.r_[:num_frequent][network_mask]
    
    # Compute offsets
    num_frequent_per_model = num_frequent // num_gpus
    network_cache_indices_offsets = np.zeros(num_gpus + 1, dtype=np.int32)
    for i in range(num_gpus):
        network_cache_indices_offsets[i] = bisect_left(network_cache_indices, i * num_frequent_per_model)
    network_cache_indices_offsets[num_gpus] = network_cache_indices.size

    return network_cache_indices, network_cache_indices_offsets, network_mask

In [ ]:
network_cache_indices = {}
network_cache_indices_offsets = {}
network_cache_masks = {}
for node_ in nodes:
    for gpu_ in node_.gpus:
        node_id = node_.node_id
        gpu_id = gpu_.gpu_id
        idx, off, mask = calculate_network_cache_indices(batch, node_id, gpu_id)
        network_cache_indices[(node_id, gpu_id)] = idx
        network_cache_indices_offsets[(node_id, gpu_id)] = off
        network_cache_masks[(node_id, gpu_id)] = mask

## Generate arrays

In [ ]:
print(list(np.reshape(category_location, 2*num_categories)))

In [ ]:
print(list(category_frequent_index))

In [ ]:
print(list(batch))

In [ ]:
for key in sorted(model_indices.keys()):
    print("{%s}," % ','.join(map(str, model_indices[key])))

In [ ]:
for key in sorted(model_indices_offsets.keys()):
    print("{%s}," % ','.join(map(str, model_indices_offsets[key])))

In [ ]:
for key in sorted(network_indices.keys()):
    print("{%s}," % ','.join(map(str, network_indices[key])))

In [ ]:
for key in sorted(network_indices_offsets.keys()):
    print("{%s}," % ','.join(map(str, network_indices_offsets[key])))

In [ ]:
for key in sorted(frequent_sample_indices.keys()):
    print("{%s}," % ','.join(map(str, frequent_sample_indices[key])))

In [ ]:
for key in sorted(model_indices.keys()):
     _, gpu = get_node_gpu(key[0], key[1])
     print("{%s}," % ','.join(map(str, gpu.embedding_output.reshape(gpu.embedding_output.size))))

In [ ]:
for key in sorted(model_cache_indices.keys()):
    print("{%s}," % ','.join(map(str, model_cache_indices[key])))

In [ ]:
for key in sorted(model_cache_indices_offsets.keys()):
    print("{%s}," % ','.join(map(str, model_cache_indices_offsets[key])))

In [ ]:
for key in sorted(model_indices.keys()):
     _, gpu = get_node_gpu(key[0], key[1])
     print("{%s}," % ','.join(map(str, gpu.a2a_fwd_message_buf.reshape(gpu.a2a_fwd_message_buf.size))))

In [ ]:
for key in sorted(network_cache_indices.keys()):
    print("{%s}," % ','.join(map(str, network_cache_indices[key])))

In [ ]:
for key in sorted(network_cache_indices_offsets.keys()):
    print("{%s}," % ','.join(map(str, network_cache_indices_offsets[key])))

In [ ]:
for key in sorted(network_cache_masks.keys()):
    print("{%s}," % ','.join(map(lambda x: str(int(x)), network_cache_masks[key])))